In [ ]:
import spacy
from collections import Counter

In [ ]:
nlp = spacy.load("en_core_web_sm")

def summarize(text, num_sentences=10):
    #Process the text throught the spacy npl
    doc = nlp(text)

    #get each token in the doc where it a word and isn't a common word like "the", "a", or "is"
    words = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop]
    #Create a counter for each of the words to find the most common
    word_freq = Counter(words)

    #Sort by most common words
    max_freq = max(word_freq.values(), default=1)
    #normalize it to give it a score of most frequent words scale count to 0-1(Adds a weight)
    for word in word_freq:
        word_freq[word] = word_freq[word] / max_freq

    #Score sentence dictionary,( Basic Hashmap frequency algorithm)
    sentence_scores = {}
    # go through each sentence of each word in the doc again 
    for sent in doc.sents:
        for word in sent:
            if word.text.lower() in word_freq: # if the word is in the freq list add it's weight
                sentence_scores[sent] = sentence_scores.get(sent, 0) + word_freq[word.text.lower()]

    #get the top sentences with the highest score
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]

    summary = " ".join([sent.text for sent in summary_sentences])

    return summary

In [ ]:
#!pip install torch
#!pip install transformers
from transformers import pipeline

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
### Had to use on Google Collab because it was to intense on my PC, but functionally works the same

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = """
WASHINGTON (AP) — President Donald Trump began imposing higher import taxes on dozens of countries Thursday just as the economic fallout of his monthslong tariff threats has begun to cause visible damage to the U.S. economy.

Just after midnight, goods from more than 60 countries and the European Union became subject to tariff rates of 10% or higher. Products from the EU, Japan and South Korea are taxed at 15%, while imports from Taiwan, Vietnam and Bangladesh are taxed at 20%. Trump also expects the EU, Japan and South Korea to invest hundreds of billions of dollars in the United States.

“I think the growth is going to be unprecedented,” Trump said Wednesday. He said the U.S. was “taking in hundreds of billions of dollars in tariffs,” but did not provide a specific figure for revenues because “we don’t even know what the final number is” regarding the rates.

Despite the uncertainty, the White House is confident that the onset of his tariffs will provide clarity about the path for the world’s largest economy. Now that companies understand the direction the U.S. is headed, the Republican administration believes it can ramp up new investments and jump-start hiring in ways that can rebalance America as a manufacturing power.

So far, however, there are signs of self-inflicted wounds to the U.S. as companies and consumers brace for the impact of the new taxes.

Hiring began to stall, inflationary pressures crept upward and home values in key markets started to decline after the initial tariff rollout in April, said John Silvia, CEO of Dynamic Economic Strategy.

“A less productive economy requires fewer workers,” Silvia said. “But there is more, the higher tariff prices lower workers’ real wages. The economy has become less productive, and firms cannot pay the same real wages as before. Actions have consequences.”

Many economists say the risk is that the American economy is steadily eroded.

“It’s going to be fine sand in the gears and slow things down,” said Brad Jensen, a professor at Georgetown University.

Trump has promoted the tariffs as a way to reduce America’s persistent trade deficit. But importers tried to avoid the taxes by bringing in more goods before the tariffs took effect. As a result, the $582.7 billion trade imbalance for the first half of the year was 38% higher than in 2024. Total construction spending has dropped 2.9% over the past year.

The economic pain is not confined to the U.S.

Germany, which sends 10% of its exports to the U.S. market, saw industrial production sag 1.9% in June as Trump’s earlier rounds of tariffs took hold. “The new tariffs will clearly weigh on economic growth,” said Carsten Brzeski, global chief of macro for ING bank.
"""
summary = summarizer(summarize(text), max_length=120, min_length=50, so_sample=False)
print(summary[0]['summary_text'])